In [1]:
from pomegranate import *

### 三门问题

In [2]:
#三门问题
#构建条件概率列表
ConditionalProbabilityList = []
for first in ['A', 'B', 'C']:
    for second in ['A', 'B', 'C']:
        for third in ['A', 'B', 'C']:
            pro = 0.0 if (third == first or third == second) else (1.0 if first != second else 0.5)
            ConditionalProbabilityList.append([first, second, third, pro])

In [3]:
len(ConditionalProbabilityList)

27

In [4]:
#初始化每个节点的分布
guest = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})
prize = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})
monty = ConditionalProbabilityTable(ConditionalProbabilityList,[guest, prize])
s1 = State(guest, name='guest')
s2 = State(prize, name='prize')
s3 = State(monty, name='monty')

In [5]:
#初始化贝叶斯网络，加入节点
model = BayesianNetwork("Monty Hall Problem")
model.add_states(s1,s2,s3)

In [6]:
#加入边
model.add_transition(s1, s3)
model.add_transition(s2, s3)
model.bake() #加入边结束

In [9]:
print('三门问题求解——')
print("P(['A','C','B']) =", model.probability(['A', 'C', 'B']))
print("P(['A','C','A']) =", model.probability(['A', 'C', 'A']))

三门问题求解——
P(['A','C','B']) = 0.11111111111111109
P(['A','C','A']) = 0.0


### Burglary问题

In [11]:
#Burglary问题
#初始化每个节点的分布
Burglary = DiscreteDistribution({'B': 0.001, '~B': 0.999})
Earthquake = DiscreteDistribution({'E': 0.002, '~E': 0.998})
Alarm = ConditionalProbabilityTable([['B','E','A',0.95],
                                     ['B','E','~A',0.05],
                                     ['B','~E','A',0.94],
                                     ['B','~E','~A',0.06],
                                     ['~B','E','A',0.29],
                                     ['~B','E','~A',0.71],
                                     ['~B','~E','A',0.001],
                                     ['~B','~E','~A',0.999]], [Burglary, Earthquake])
Johncalls = ConditionalProbabilityTable([['A', 'J', 0.9],
                                         ['A', '~J', 0.1],
                                         ['~A', 'J', 0.05],
                                         ['~A', '~J', 0.95]],[Alarm])
Marycalls = ConditionalProbabilityTable([['A', 'M', 0.7],
                                         ['A', '~M', 0.3],
                                         ['~A', 'M', 0.01],
                                         ['~A', '~M', 0.99]],[Alarm])
s1 = State(Burglary, name='Burglary')
s2 = State(Earthquake, name='Earthquake')
s3 = State(Alarm, name='Alarm')
s4 = State(Johncalls, name='Johncalls')
s5 = State(Marycalls, name='Marycalls')

In [12]:
#初始化贝叶斯网络，加入各节点
model = BayesianNetwork("Burglary problem")
model.add_states(s1,s2,s3,s4,s5)

In [13]:
#加入边
model.add_transition(s1,s3)
model.add_transition(s2,s3)
model.add_transition(s3,s4)
model.add_transition(s3,s5)
model.bake()

In [14]:
#计算联合概率
model_var_dict = {'Burglary':['B', '~B'], 'Earthquake':['E','~E'], 
                  'Alarm':['A','~A'], 'Johncalls':['J','~J'], 
                  'Marycalls':['M','~M']}
def calc_joint_pro(model, **var_dict):
    for var in model_var_dict.keys():
        if var_dict.get(var, None) == None:
            res = 0.
            temp_dict = var_dict.copy()
            for value in model_var_dict[var]:
                temp_dict[var] = value
                res += calc_joint_pro(model, **temp_dict)
            return res
    return model.probability([var_dict[var] for var in model_var_dict.keys()])

In [19]:
p1 = calc_joint_pro(model, **{'Johncalls': 'J', 'Marycalls': 'M'})
p2 = calc_joint_pro(model, **{'Burglary': 'B', 'Earthquake': 'E', 'Alarm': 'A', 'Johncalls': 'J', 'Marycalls': 'M'})
p3 = calc_joint_pro(model, **{'Alarm': 'A', 'Johncalls': 'J', 'Marycalls': 'M'})/calc_joint_pro(model, **{'Johncalls': 'J', 'Marycalls': 'M'})
p4 = calc_joint_pro(model, **{'Burglary': '~B', 'Johncalls': 'J', 'Marycalls': '~M'}) / calc_joint_pro(model, **{'Burglary': '~B'})

In [18]:
print('Burglary问题求解——')
print("P(JohnCalls, MaryCalls) =", p1)
print("P(Burglary, Earthquake, Alarm, JohnCalls, MaryCalls) =", p2)
print("P(Alarm | JohnCalls, MaryCalls) =", p3)
print("P(JohnCalls, ¬ MaryCalls | ¬ Burglary) =", p4)

Burglary问题求解——
P(JohnCalls, MaryCalls) = 0.0020841002390000014
P(Burglary, Earthquake, Alarm, JohnCalls, MaryCalls) = 1.1969999999999995e-06
P(Alarm | JohnCalls, MaryCalls) = 0.7606920388631077
P(JohnCalls, ¬ MaryCalls | ¬ Burglary) = 0.049847948999999996


### Diagnosing问题

In [20]:
from pomegranate import *
PatientAge = DiscreteDistribution({'0-30':0.1, '31-65':0.3, '65+': 0.6})
CTScanResult = DiscreteDistribution({'Ischemic':0.7, 'Hemmorraghic':0.3})
MRIScanResult = DiscreteDistribution({'Ischemic':0.7, 'Hemmorraghic':0.3})
Anticoagulants = DiscreteDistribution({'Used':0.5, 'Not used': 0.5})

StrokeType = ConditionalProbabilityTable([['Ischemic','Ischemic','Ischemic',0.8],
                                          ['Ischemic','Ischemic','Hemmorraghic',0.0],
                                          ['Ischemic','Ischemic','Mimic',0.2],
                                          ['Ischemic','Hemmorraghic','Ischemic',0.5],
                                          ['Ischemic','Hemmorraghic','Hemmorraghic',0.4],
                                          ['Ischemic','Hemmorraghic','Mimic',0.1],
                                          ['Hemmorraghic','Ischemic','Ischemic',0.5],
                                          ['Hemmorraghic','Ischemic','Hemmorraghic',0.4],
                                          ['Hemmorraghic','Ischemic','Mimic',0.1],
                                          ['Hemmorraghic','Hemmorraghic','Ischemic',0.0],
                                          ['Hemmorraghic','Hemmorraghic','Hemmorraghic',0.9],
                                          ['Hemmorraghic','Hemmorraghic','Mimic',0.1]],[CTScanResult, MRIScanResult])

Mortality = ConditionalProbabilityTable([['Ischemic','Used','False',0.28],
                                         ['Ischemic','Used','True',0.72],
                                         ['Ischemic','Not used','False',0.56],
                                         ['Ischemic','Not used','True',0.44],
                                         ['Hemmorraghic','Used','False',0.99],
                                         ['Hemmorraghic','Used','True',0.01],
                                         ['Hemmorraghic','Not used','False',0.58],
                                         ['Hemmorraghic','Not used','True',0.42],
                                         ['Mimic','Used','False',0.1],
                                         ['Mimic','Used','True',0.9],
                                         ['Mimic','Not used','False',0.05],
                                         ['Mimic','Not used','True',0.95]],[StrokeType, Anticoagulants])

Disability = ConditionalProbabilityTable([['Ischemic','0-30','Negligible',0.8],
                                          ['Ischemic','0-30','Moderate',0.1],
                                          ['Ischemic','0-30','Severe',0.1],
                                          ['Hemmorraghic','0-30','Negligible',0.7],
                                          ['Hemmorraghic','0-30','Moderate',0.2],
                                          ['Hemmorraghic','0-30','Severe',0.1],
                                          ['Mimic','0-30','Negligible',0.9],
                                          ['Mimic','0-30','Moderate',0.05],
                                          ['Mimic','0-30','Severe',0.05],
                                    
                                          ['Ischemic','31-65','Negligible',0.6],
                                          ['Ischemic','31-65','Moderate',0.3],
                                          ['Ischemic','31-65','Severe',0.1],
                                          ['Hemmorraghic','31-65','Negligible',0.5],
                                          ['Hemmorraghic','31-65','Moderate',0.4],
                                          ['Hemmorraghic','31-65','Severe',0.1],
                                          ['Mimic','31-65','Negligible',0.4],
                                          ['Mimic','31-65','Moderate',0.3],
                                          ['Mimic','31-65','Severe',0.3],
                                          
                                          ['Ischemic','65+','Negligible',0.3],
                                          ['Ischemic','65+','Moderate',0.4],
                                          ['Ischemic','65+','Severe',0.3],
                                          ['Hemmorraghic','65+','Negligible',0.2],
                                          ['Hemmorraghic','65+','Moderate',0.2],
                                          ['Hemmorraghic','65+','Severe',0.6],
                                          ['Mimic','65+','Negligible',0.1],
                                          ['Mimic','65+','Moderate',0.1],
                                          ['Mimic','65+','Severe',0.8]],[StrokeType,PatientAge])

In [21]:
s1 = State(PatientAge, name='PatientAge')
s2 = State(CTScanResult, name='CTScanResult')
s3 = State(MRIScanResult, name='MRIScanResult')
s4 = State(Anticoagulants, name='Anticoagulants')
s5 = State(StrokeType, name='StrokeType')
s6 = State(Mortality, name='Mortality')
s7 = State(Disability, name='Disability')

In [22]:
#初始化贝叶斯网络，加入各节点
model = BayesianNetwork("Diagnose problem")
model.add_states(s1,s2,s3,s4,s5,s6,s7)

model.add_transition(s2,s5)
model.add_transition(s3,s5)
model.add_transition(s5,s6)
model.add_transition(s4,s6)
model.add_transition(s5,s7)
model.add_transition(s1,s7)
model.bake()

In [23]:
model_var_dict = {'PatientAge':['0-30','31-65','65+'],
                  'CTScanResult': ['Ischemic','Hemmorraghic'],
                  'MRIScanResult':['Ischemic','Hemmorraghic'],
                  'Anticoagulants':['Used', 'Not used'],
                  'StrokeType':['Ischemic','Hemmorraghic','Mimic'],
                  'Mortality': ['False', 'True'],
                  'Disability': ['Negligible','Moderate', 'Severe']}
def calc_joint_pro(model, **var_dict):
    for var in model_var_dict.keys():
        if var_dict.get(var, None) == None:
            res = 0.
            temp_dict = var_dict.copy()
            for value in model_var_dict[var]:
                temp_dict[var] = value
                res += calc_joint_pro(model, **temp_dict)
            return res
    return model.probability([var_dict[var] for var in model_var_dict.keys()])

In [24]:
p1 = calc_joint_pro(model, **{'Mortality':'True','PatientAge':'0-30', 'CTScanResult':'Ischemic'}) \
    / calc_joint_pro(model, **{'PatientAge':'0-30', 'CTScanResult':'Ischemic'})
p2 = calc_joint_pro(model, **{'Disability':'Severe','PatientAge':'65+','MRIScanResult':'Ischemic'}) \
    / calc_joint_pro(model, **{'PatientAge':'65+','MRIScanResult':'Ischemic'})
p3 = calc_joint_pro(model, **{'StrokeType':'Mimic','PatientAge':'65+', 
                         'CTScanResult':'Hemmorraghic','MRIScanResult':'Ischemic'}) \
    / calc_joint_pro(model, **{'PatientAge':'65+', 
                         'CTScanResult':'Hemmorraghic','MRIScanResult':'Ischemic'})
p4 = calc_joint_pro(model, **{'Mortality':'False','PatientAge':'0-30', 
                         'Anticoagulants':'Used', 'StrokeType':'Mimic'}) \
    / calc_joint_pro(model, **{'PatientAge':'0-30', 
                         'Anticoagulants':'Used', 'StrokeType':'Mimic'})
p5 = calc_joint_pro(model, **{'PatientAge':'0-30', 'CTScanResult':'Ischemic', 
                         'MRIScanResult':'Hemmorraghic', 'Anticoagulants':'Used', 
                         'StrokeType':'Mimic', 'Disability':'Severe', 'Mortality':'False'})

In [29]:
print('Diagnosing问题求解——')
print("P(Mortality='True'|PatientAge='0-30',CTScanResult='Ischemic Stroke') =", p1)
print("P(Disability=' Severe'|PatientAge='65+',MRIScanResult='Ischemic Stroke') =", p2)
print("P(StrokeType='Stroke Mimic'|PatientAge='65+',CTScanResult='Hemmorraghic Stroke',MRIScanResult='Ischemic Stroke') =", p3)
print("P(Mortality='False'|PatientAge='0-30',Anticoagulants='Used,StrokeType='Stroke Mimic') =", p4)
print("P(PatientAge='0-30',CTScanResult='Ischemic Stroke',MRIScanResult='Hemmorraghic Stroke',\n \
 Anticoagulants='Used',StrokeType='Stroke Mimic',Disability='Severe',Mortality='False') =", p5)

Diagnosing问题求解——
P(Mortality='True'|PatientAge='0-30',CTScanResult='Ischemic Stroke') = 0.5948500000000002
P(Disability=' Severe'|PatientAge='65+',MRIScanResult='Ischemic Stroke') = 0.4210000000000001
P(StrokeType='Stroke Mimic'|PatientAge='65+',CTScanResult='Hemmorraghic Stroke',MRIScanResult='Ischemic Stroke') = 0.10000000000000007
P(Mortality='False'|PatientAge='0-30',Anticoagulants='Used,StrokeType='Stroke Mimic') = 0.09999999999999995
P(PatientAge='0-30',CTScanResult='Ischemic Stroke',MRIScanResult='Hemmorraghic Stroke',
  Anticoagulants='Used',StrokeType='Stroke Mimic',Disability='Severe',Mortality='False') = 5.250000000000009e-06


In [7]:
calc_joint_pro(model, **{'Mortality':'True','PatientAge':'0-30', 'CTScanResult':'Ischemic'}) \
    / calc_joint_pro(model, **{'PatientAge':'0-30', 'CTScanResult':'Ischemic'})

0.5948500000000002

In [8]:
calc_joint_pro(model, **{'Disability':'Severe','PatientAge':'65+','MRIScanResult':'Ischemic'}) \
    / calc_joint_pro(model, **{'PatientAge':'65+','MRIScanResult':'Ischemic'})

0.4210000000000001

In [9]:
calc_joint_pro(model, **{'StrokeType':'Mimic','PatientAge':'65+', 
                         'CTScanResult':'Hemmorraghic','MRIScanResult':'Ischemic'}) \
    / calc_joint_pro(model, **{'PatientAge':'65+', 
                         'CTScanResult':'Hemmorraghic','MRIScanResult':'Ischemic'})

0.10000000000000007

In [10]:
calc_joint_pro(model, **{'Mortality':'False','PatientAge':'0-30', 
                         'Anticoagulants':'Used', 'StrokeType':'Mimic'}) \
    / calc_joint_pro(model, **{'PatientAge':'0-30', 
                         'Anticoagulants':'Used', 'StrokeType':'Mimic'})

0.09999999999999995

In [11]:
calc_joint_pro(model, **{'PatientAge':'0-30', 'CTScanResult':'Ischemic', 
                         'MRIScanResult':'Hemmorraghic', 'Anticoagulants':'Used', 
                         'StrokeType':'Mimic', 'Disability':'Severe', 'Mortality':'False'})

5.250000000000009e-06